In [105]:
import face_recognition
import cv2
from uuid import  uuid4
import re
import numpy as np
from tqdm import  tqdm
import os

In [106]:
def faces(path):
    # load the file 
    image = face_recognition.load_image_file(path)
    #resize 
    # image = cv2.resize(image)
    # recognize the face locations 
    face_locations = face_recognition.face_locations(image)
    if len(face_locations)==0:
        return None,None
    # extract the faces 
    faces = []
    for location in face_locations:
        top, right, bottom, left = location
        face = image[top:bottom, left:right]
        faces.append(face)

    return faces,face_locations


In [107]:
def get_counter(dir="output"):
    createDir_if_not_exists(dir)
    files = os.listdir(dir)
    
    if len(files) == 0:
        return 1  # If no files, start with person1
    
    # Regular expression to match 'person' followed by an integer (e.g., person1, person12, etc.)
    pattern = re.compile(r"person(\d+)")
    
    max_number = 0
    
    for file in files:
        match = pattern.match(file)
        if match:
            # Extract the number from the file name
            number = int(match.group(1))
            max_number = max(max_number, number)
    
    # Return the next counter
    return max_number + 1
person_count = get_counter()

In [108]:
def save(face,filepath,resize=None):
    # resize the image 
    if not resize is None:
        if isinstance(resize,tuple):
            face=cv2.resize(face, dsize=resize,)
        if isinstance(resize,int):
            face=cv2.resize(face,dsize=(resize,resize))
    cv2.imwrite(filepath,face)
    return

In [109]:
def generate_filepath(extension='jpg'):
    file_name = str(uuid4())
    if re.findall(r'\.',extension):
        file_name = file_name+extension
    else:
        file_name = f"{file_name}.{extension}"
    return file_name


In [110]:
def createDir_if_not_exists(dir):
    import os 
    # check the directory is exists or not
    if not os.path.exists(dir):
        os.mkdir(path=dir)
    return 

In [111]:
import numpy as np

def getEmbedding(face):
    # Convert to RGB as face_recognition expects the image in RGB
    convert_color = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    
    # Get the face encoding (embedding)
    embedding = face_recognition.face_encodings(convert_color)
    
    if len(embedding) == 0:
        return None
    
    return embedding[0]

In [112]:
def save_face(face=None,dir='default',base='output'):
    base = 'output'
    createDir_if_not_exists(base)
    createDir_if_not_exists(f"{base}/{dir}")
    # check the faces is avalilable or not 
    if faces is None:
        return
    filename = f"output/{dir}/{generate_filepath()}"
    # convert the image into RGB to BGR 
    face = cv2.cvtColor(face,cv2.COLOR_RGB2BGR)
    # save the file
    save(face,filename,resize=(64,64))
    print("Face saved")
    return

In [113]:
from qdrant_client import QdrantClient,models
from qdrant_client.models import NamedVector,VectorParams,Distance,PointStruct
threshold = 0.93
def connect_qdrant():
    # Create a QdrantClient instance, specifying the host and port
    client = QdrantClient(host='localhost', port=6333)  # Adjust host and port as needed
    return client
client = connect_qdrant()


In [114]:
def create_collection_if_not_exists(collection_name="faces", vector_size=128,):
    # Check if the collection already exists
    collections = client.get_collections()

    # If the collection does not exist, create it
    if collection_name not in [col.name for col in collections.collections]:
        print(f"Collection '{collection_name}' does not exist. Creating a new one.")
        
        # Create the collection
        client.create_collection(
            collection_name=collection_name,
            vectors_config=models.VectorParams(size=vector_size, distance=models.Distance.COSINE),
        )

        print(f"Collection '{collection_name}' created successfully.")
    else:
        print(f"Collection '{collection_name}' already exists.")

In [115]:
create_collection_if_not_exists()

Collection 'faces' already exists.


In [116]:
def compare_embedding(embedding,collection_name="faces",top_k=1):
    results = client.search(
        collection_name=collection_name,      # Collection to search in
        query_vector=embedding,                # The embedding you want to compare
        limit=top_k,                           # Number of nearest neighbors to return
    )
    # Return the search results
    return results

# add the new embedding 
def add_data(embedding,collection_name="faces",name="default"):
    point_id = str(uuid4())  # Replace with actual logic to generate a unique ID, like using a counter or UUID.
    # Create a PointStruct with embedding and metadata
    point = PointStruct(id=point_id, vector=embedding, payload={"name": name})
    # Add the new point to the collection
    client.upsert(collection_name=collection_name, points=[point])
    print("Added")    

In [117]:
def handle_new_face(face,dir_name,embedding):
    global person_count
    dir = f"{dir_name}{person_count}"
    save_face(face=face,dir=dir)
    add_data(embedding=embedding,name=dir)
    person_count+=1
    return person_count


In [118]:
dir_name = "person"

def process_faces(face_locations,faces=None,):
    if faces is None:
        print("No faces")
        return
    
    for face in tqdm(faces,desc="Processing face"):
        embedding = getEmbedding(face)
        if embedding is None:
            continue
        compare_result = compare_embedding(embedding)
        if len(compare_result)==0:
            handle_new_face(face,dir_name,embedding=embedding)
            # add new data 
            continue
        # get the name and score 
        score = compare_result[0].score  
        print(f"Score:{score}")
        name = compare_result[0].payload["name"] 
        if score>threshold:
            #add save the image file in the name directory
            save_face(face,dir=name)
        else:
            handle_new_face(face,dir_name,embedding=embedding)
            
        embedding=None


In [119]:
def read_all_files_in_directory(dir="input"):
    all_files = []
    
    # Walk through the directory and its subdirectories
    for root, dirs, files in os.walk(dir):
        for file in files:
            all_files.append(os.path.join(root, file))  # Add full file path
    
    return all_files

In [120]:
def process(paths):
    for path in tqdm(paths,"Processing file"):
        faces_roi,locations = faces(path)
        if faces_roi is None or  len(faces_roi)==0:
            continue
        process_faces(face_locations=locations,faces=faces_roi)
        

In [121]:
paths = read_all_files_in_directory()

In [ ]:

process(paths)

Processing file:   0%|          | 0/1312 [00:00<?, ?it/s]

Face saved
Added


Score:0.83515614
Face saved
Added


Processing file:   0%|          | 1/1312 [00:14<5:23:00, 14.78s/it]

Score:0.8986392
Face saved
Added


...

Score:0.9798441
Face saved


Processing file:   4%|▍         | 59/1312 [14:08<5:08:46, 14.79s/it]

Score:0.97293276
Face saved


Processing file:   5%|▍         | 60/1312 [14:23<5:07:08, 14.72s/it]

Score:0.9087075
Face saved
Added
